In [1]:
import tensorflow as tf
from tensorflow import keras

# Load the model from the .h5 file
model_predict_codition = keras.models.load_model('model_predict_condition.h5')

model_predict_precip = keras.models.load_model('model_predict_precip.h5')
model_predict_temp= keras.models.load_model('model_predict_temp.h5')




In [2]:
model_predict_codition.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 12, 50)            12600     
                                                                 
 dense_2 (Dense)             (None, 12, 32)            1632      
                                                                 
 flatten_1 (Flatten)         (None, 384)               0         
                                                                 
 dense_3 (Dense)             (None, 3)                 1155      
                                                                 
Total params: 15,387
Trainable params: 15,387
Non-trainable params: 0
_________________________________________________________________


In [2]:
import pandas as pd
import requests
import sys
import json
from datetime import datetime
import pandas as pd
import requests
import sys
import json
from datetime import datetime,timedelta
import numpy as np
# #Đọc dataset
data = pd.read_csv('data_predict.csv')
temp_humidity=pd.read_csv('temp_humidity.csv')


now = datetime.now()
ngay_hien_tai = now.strftime('%Y-%m-%d')
gio_hien_tai = now.strftime('%H')

# Lùi ngày và giờ đi 12 giờ
ngay_truoc = (now - timedelta(hours=12)).strftime('%Y-%m-%d')
gio_truoc = (now - timedelta(hours=12)).strftime('%H')
if ngay_truoc==ngay_hien_tai:
    ngay_truoc = (now - timedelta(hours=24)).strftime('%Y-%m-%d')
# Tạo URL để gửi yêu cầu API thời tiết
url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Da%20Nang/{ngay_truoc}/{ngay_hien_tai}?unitGroup=metric&include=hours&key=MD5UMA5QNPNSM4FM7W5ZV6DL6&contentType=json"

# Gửi yêu cầu API
response = requests.get(url)
if response.status_code != 200:
    print('Unexpected Status code: ', response.status_code)
    sys.exit()

# Parse kết quả nhận được từ API
jsonData = response.json()

#Nối vào dataset
for i in range(12):
  hour_index = int(gio_hien_tai) - 12+i+1
  days = None
  if hour_index < 0:
      hour_index += 24
      days = jsonData['days'][0]['hours'][hour_index]
  else:
      days = jsonData['days'][1]['hours'][hour_index]

  new_data = [ days['dew'], days['precip'], days['precipprob'], days['windspeed'], days['winddir'], days['cloudcover'], days['visibility'], days['solarradiation'],days['solarenergy'], days['uvindex'], days['conditions']]
  temp_hu=[days['temp'], days['humidity']]

  new_data.insert(0, temp_hu[0])
  new_data.insert(2, temp_hu[1])

  data.loc[len(data)+1] = new_data
  temp_humidity.loc[len(temp_humidity)+1]=temp_hu

data=data.tail(12)
temp_humidity=temp_humidity.tail(12)


X = pd.DataFrame(data[['temp','dew','humidity','precip','windspeed','winddir','cloudcover','visibility','solarradiation','solarenergy','conditions','precipprob']])



temp_humidity.to_csv('temp_humidity.csv', index=False) 

data.to_csv('data_predict.csv',index=False)

X['conditions'] = X['conditions'].replace('Clear', 0)
X['conditions'] = X['conditions'].replace('Partially cloudy', 1)
X['conditions'] = X['conditions'].replace('Overcast', 2)
X['conditions'] = X['conditions'].replace('Rain, Partially cloudy', 3)
X['conditions'] = X['conditions'].replace('Rain, Overcast', 4)
X.loc[X['precipprob']==100,'precipprob']=1

X_test_ = X[-12:]

X_test = X_test_.to_numpy().reshape(1,12,12)

y_precip_predict = model_predict_precip.predict(X_test).flatten()
y_condition_predict = model_predict_codition.predict(X_test).flatten()
y_temp_predict = model_predict_temp.predict(X_test).flatten()

y_precip_predict[y_precip_predict < 0] +=0.05
y_precip_predict = np.round(y_precip_predict * 100, decimals=1)
y_temp_predict = np.round(y_temp_predict+2, decimals=0)

y_precip_predict = y_precip_predict.flatten()
y_condition_predict = y_condition_predict.flatten()
y_temp_predict=y_temp_predict.flatten()

print(y_condition_predict)
y_condition_predict = pd.Series((y_condition_predict).round().astype(int))
y_condition_predict = y_condition_predict.replace(0, 'Clear')
y_condition_predict = y_condition_predict.replace(1, 'Partially cloudy')
y_condition_predict = y_condition_predict.replace(2, 'Overcast')
y_condition_predict = y_condition_predict.replace(3, 'Rain, Partially cloudy')
y_condition_predict = y_condition_predict.replace(4, 'Rain, Overcast')


data = {
    'precipprob': y_precip_predict,
    'condition': list(y_condition_predict),
    'temp': y_temp_predict
}

df = pd.DataFrame(data)


from tabulate import tabulate
import pandas as pd
X_test_view = pd.DataFrame(X_test_)
X_test_view=X_test_view.drop(['conditions','precipprob'],axis=1)

from datetime import datetime, timedelta

end_time = datetime.now()
start_time = end_time - timedelta(hours=11)
new_index = pd.date_range(start=start_time, end=end_time, freq='H')


st_time=datetime.now()+timedelta(hours=1)
e_time=st_time+timedelta(hours=2)
new_index_df = pd.date_range(start=st_time, end=e_time, freq='H')
# # Tạo DataFrame với index là các timestamps
X_test_view=X_test_view.set_index(new_index)
X_test_view.index = X_test_view.index.strftime('%Y-%m-%d %H:00')

df=df.set_index(new_index_df)
df.index = df.index.strftime('%Y-%m-%d %H:00')


print(tabulate(df , headers="keys", tablefmt="fancy_grid", numalign="left"))
print(tabulate(X_test_view, headers="keys", tablefmt="fancy_grid", numalign="left"))


1/1 [==============================] - 1s 690ms/step
[0.83338267 0.9802938  1.0644246 ]
╒══════════════════╤══════════════╤══════════════════╤════════╕
│                  │ precipprob   │ condition        │ temp   │
╞══════════════════╪══════════════╪══════════════════╪════════╡
│ 2023-06-07 16:00 │ 2.3          │ Partially cloudy │ 31     │
├──────────────────┼──────────────┼──────────────────┼────────┤
│ 2023-06-07 17:00 │ 0            │ Partially cloudy │ 30     │
├──────────────────┼──────────────┼──────────────────┼────────┤
│ 2023-06-07 18:00 │ 2.1          │ Partially cloudy │ 29     │
╘══════════════════╧══════════════╧══════════════════╧════════╛
╒══════════════════╤════════╤═══════╤════════════╤══════════╤═════════════╤═══════════╤══════════════╤══════════════╤══════════════════╤═══════════════╕
│                  │ temp   │ dew   │ humidity   │ precip   │ windspeed   │ winddir   │ cloudcover   │ visibility   │ solarradiation   │ solarenergy   │
╞══════════════════╪════════╪═

In [3]:
a = now.strftime('%M')


In [4]:
pre_value=[]
con_value=[]

for index, row in df.iterrows():
    pre_value .append( row['precipprob'])
    con_value .append(row['condition'])
